## Retrieve review and rebuttal threads

Reviewers and authors often thread their reviews and responses (possibly due to character limits?)

The code below retrieves
1. Reviews
2. The longest unbroken thread of replies by the same reviewer to each review
3. Rebuttals (replies by the authors to an official review)
4. The longest unbroken thread of replies by the authors to their first reply to the review

In [1]:
import collections
from datetime import datetime
import openreview
import os
import tqdm


# Change these values according to your needs
INVITATION = 'ICLR.cc/2021/Conference/-/Blind_Submission'
LIMIT = 10 # Number of papers to build timelines for

def get_replies(note, discussion_notes):
  return [x for x in discussion_notes if x.replyto == note.id]

def get_longest_thread(top_note, discussion_notes):
  threads = [[top_note]]
  while True:
    new_threads = []
    for thread in threads:
      thread_last = thread[-1]
      candidates = [c for c in get_replies(thread_last, discussion_notes) if c.signatures == thread_last.signatures]
      for candidate in candidates:
        new_threads.append(thread + [candidate])
    if not new_threads:
      break
    threads = new_threads
  return max(threads, key=lambda x:len(x))
    

# A client is required for any OpenReview API actions
guest_client = openreview.Client(baseurl='https://api.openreview.net')

def get_review_threads(discussion_notes):
  reviews = []
  for note in discussion_notes:
    if note.replyto == note.forum and 'review' in note.content:
      reviews.append(get_longest_thread(note, discussion_notes))
  return reviews

def get_rebuttal_threads(review_threads, discussion_notes):
  review_note_ids = [t[0].id for t in review_threads]
  rebuttals = []
  for note in discussion_notes:
    if note.replyto in review_note_ids and 'Authors' in note.signatures[0]:
      rebuttals.append(get_longest_thread(note, discussion_notes))
  return rebuttals
  
    

review_lens = collections.Counter()
rebuttal_lens = collections.Counter()
event_list = []
for i, forum_note in tqdm.tqdm(enumerate(openreview.tools.iterget_notes(
        guest_client, invitation=INVITATION))):
  discussion_notes = guest_client.get_notes(forum=forum_note.id)
  review_threads = get_review_threads(discussion_notes)
  for r in review_threads:
    review_lens[len(r)] += 1
  rebuttal_threads = get_rebuttal_threads(review_threads, discussion_notes)
  for r in rebuttal_threads:
    rebuttal_lens[len(r)] += 1

print("Review lengths (number of notes)")
print(review_lens)
print("Rebuttal lengths (number of notes)")
print(rebuttal_lens)


2595it [11:36,  3.72it/s]

Review lengths (number of notes)
Counter({1: 9948, 2: 76, 3: 2})
Rebuttal lengths (number of notes)
Counter({1: 9386, 2: 1046, 3: 170, 4: 29, 5: 7, 6: 2})
